# LangChain: Q&A over Documents

An example might be a tool that would allow you to query a product catalog for items of interest.

In [1]:
#pip install --upgrade langchain

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [2]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [8]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings



In [4]:
file = 'titanic.csv'
loader = CSVLoader(file_path=file)

In [5]:
from langchain.indexes import VectorstoreIndexCreator

In [6]:
#pip install docarray

     |████████████████████████████████| 270 kB 2.2 MB/s eta 0:00:01
     |████████████████████████████████| 242 kB 5.0 MB/s eta 0:00:01
     |████████████████████████████████| 87 kB 3.0 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [9]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

SyntaxError: invalid syntax (1858301364.py, line 3)

In [ ]:
query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

In [12]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

**Note**:
- The notebook uses `langchain==0.0.179` and `openai==0.27.7`
- For these library versions, `VectorstoreIndexCreator` uses `text-davinci-003` as the base model, which has been deprecated since 1 January 2024.
- The replacement model, `gpt-3.5-turbo-instruct` will be used instead for the `query`.
- The `response` format might be different than the video because of this replacement model.

In [13]:
llm_replacement_model = OpenAI(temperature=0, 
                               model='gpt-3.5-turbo-instruct')

response = index.query(query, 
                       llm = llm_replacement_model)

NameError: name 'index' is not defined

In [ ]:
display(Markdown(response))

## Step By Step

In [14]:
from langchain.document_loaders import CSVLoader
loader = CSVLoader(file_path=file)

In [15]:
docs = loader.load()

In [16]:
docs[0]

Document(metadata={'source': 'titanic.csv', 'row': 0}, page_content='Survived: 0\nPclass: 3\nName: Mr. Owen Harris Braund\nSex: male\nAge: 22\nSiblings/Spouses Aboard: 1\nParents/Children Aboard: 0\nFare: 7.25')

In [17]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

/tmp/ipykernel_11276/2047715053.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [18]:
embed = embeddings.embed_query("Hi my name is Manuel")

In [19]:
print(len(embed))

1536


In [20]:
print(embed[:5])

[-0.028231139666453162, 0.013487356447522143, -0.013936079762269567, -0.027154201103356188, -0.019846414753737034]


In [21]:
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embeddings
)

/home/manuel/Documents/ACCION_old/clients/yves_cariou_3.9/venv/lib/python3.9/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [22]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv(file)

# Display the first few rows of the DataFrame to see the main columns
df.head()

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833
2,1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.9250
3,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35.0,1,0,53.1000
4,0,3,Mr. William Henry Allen,male,35.0,0,0,8.0500


In [23]:
query = "Please tell me the oldest passenger's name who died in class 1."

In [24]:
docs = db.similarity_search(query)

In [25]:
len(docs)

4

In [26]:
docs[0]

Document(metadata={'source': 'titanic.csv', 'row': 778}, page_content='Survived: 0\nPclass: 1\nName: Mr. Milton Clyde Long\nSex: male\nAge: 29\nSiblings/Spouses Aboard: 0\nParents/Children Aboard: 0\nFare: 30')

In [27]:
retriever = db.as_retriever()

In [28]:
llm = ChatOpenAI(temperature = 0.0, model=llm_model)

In [29]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])


In [30]:
response = llm.call_as_llm(f"{qdocs} Question: tell me the oldest passenger who died in class 1") 


/tmp/ipykernel_11276/350039275.py:1: LangChainDeprecationWarning: The method `BaseChatModel.call_as_llm` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm.call_as_llm(f"{qdocs} Question: tell me the oldest passenger who died in class 1")


In [31]:
display(Markdown(response))

The oldest passenger who died in class 1 was Mr. Milton Clyde Long, who was 29 years old.

In [ ]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [ ]:
query =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."

In [ ]:
response = qa_stuff.run(query)

In [ ]:
display(Markdown(response))

In [ ]:
response = index.query(query, llm=llm)

In [ ]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings,
).from_loaders([loader])

Reminder: Download your notebook to you local computer to save your work.